# Proyecto de Recuperación de Información: Dataset Cranfield
### Motor de Búsqueda Vectorial y Probabilístico para Ingeniería Aeronáutica

**Autor:** Fabian Simbaña, Brayan Ortiz.

**Dataset:** Cranfield Collection (1400 abstracts sobre aerodinámica).

### Objetivo.
Diseñar e implementar un sistema de recuperaci´on de informaci´on que indexe un conjunto de documentos en
texto plano y permita ejecutar consultas de texto libre utilizando el modelo vectorial con vectores binarios
y ponderación TF-IDF, y el modelo probabil´ıstico BM25. El sistema debe permitir evaluar la calidad de los
resultados utilizando m´etricas est´andar como precision y recall.
### Arquitectura.
1.  **Preprocesamiento de Dominio:**
    * Normalización.
    * Limpieza de ruido "académico" específico de ingeniería (ej: *calculated, measured*).
    * Preservación de terminología técnica con guiones (ej: *quasi-linear*).
    * Stemming (Porter) para normalización morfológica.
3.  **Modelos Implementados (From Scratch):**
    * **Jaccard:** Coeficiente de similitud de conjuntos.
    * **TF-IDF:** Modelo vectorial con suavizado logarítmico ($1 + \log(tf)$).
    * **BM25:** Modelo probabilístico calibrado($k_1=1.8, b=0.9$).
4.  **Evaluación:**
    * Métricas estandarizadas: **MAP**, **Precision@10**, **Recall@10**.

In [97]:
import numpy as np
import pandas as pd
import re
import math
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from numpy.linalg import norm
import os

# =============================================================================
# 1. PREPROCESAMIENTO DE TEXTO (DOMINIO: INGENIERÍA)
# =============================================================================

# Descarga de recursos básicos
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

stemmer = PorterStemmer()

# --- A. DEFINICIÓN DE STOPWORDS ---
# Lista base de inglés
lista_stopwords_raw = set(stopwords.words('english'))

# RUIDO ESPECÍFICO DE CRANFIELD:
# Palabras metodológicas que aparecen en casi todos los papers y no discriminan.
cranfield_noise = {
    # Términos generales
    'abstract', 'paper', 'report', 'note', 'present', 'discuss', 
    'introduction', 'conclus', 'conclusion', 'result', 'obtain', 
    'shown', 'show', 'give', 'given', 'studi', 'study', 'investig', 
    'investigation', 'research', 'work', 'development', 'consider',
    
    # Verbos/Acciones (Ruido alto en ingeniería)
    'method', 'approach', 'technique', 'use', 'using', 'base', 'based',
    'determin', 'determine', 'calcul', 'calculate', 'comput', 
    'measur', 'measure', 'measurement', 'estim', 'estimate',
    'experi', 'experiment', 'experimental', 'test', 'analy', 'analysis',
    'appl', 'appli', 'application', 'compar', 'comparison',
    
    # Términos abstractos
    'effect', 'affect', 'influence', 'theori', 'theory', 'theoretical',
    'problem', 'solut', 'solution', 'case', 'approxim', 'approximate', 
    'approximation', 'condit', 'condition', 'gener', 'general',
    'valu', 'value', 'number', 'agree', 'agreement', 'data', 'time', 
    'year', 'refer', 'reference', 'equat', 'equation', 'deriv', 'derivation',
    'variou', 'various', 'possibl', 'possible', 'type'
}

lista_stopwords_raw.update(cranfield_noise)
stop_words_stemmed = set([stemmer.stem(w) for w in lista_stopwords_raw])

def procesar_texto_seguro(texto):
    """
    Pipeline de limpieza optimizado para textos técnicos:
    1. Regex: Mantiene alfanuméricos y GUIONES (-) (vital para 'steady-flow').
    2. Stemming: Reduce variaciones morfológicas.
    3. Filtrado: Elimina stopwords generales y de dominio.
    """
    if not isinstance(texto, str): return []
    
    # 1. Limpieza (Conservamos guiones internos)
    text = re.sub(pattern=r"<.*?>", repl=' ', string=texto.lower())
    text = re.sub(r"[^a-z0-9\s-]", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    tokens = text.split()
    tokens_procesados = []
    
    # 2. Normalización y Filtrado
    for t in tokens:
        # Filtramos guiones sueltos o palabras muy cortas
        if len(t) > 2 and t != '-': 
            raiz = stemmer.stem(t)
            if raiz not in stop_words_stemmed and len(raiz) > 2:
                tokens_procesados.append(raiz)
    
    return tokens_procesados

print("Configuración cargada: Preprocesamiento optimizado para Cranfield.")

Configuración cargada: Preprocesamiento optimizado para Cranfield.


In [98]:
# =============================================================================
# 2. MODELOS DE RECUPERACIÓN
# =============================================================================

# --- A. JACCARD (Similitud de Conjuntos) ---
def busqueda_jaccard(consulta, corpus, top_n=10):
    query_tokens = set(procesar_texto_seguro(consulta))
    scores = []
    for i, doc_tokens in enumerate(corpus):
        doc_set = set(doc_tokens)
        if not query_tokens or not doc_set: continue
        
        intersection = len(query_tokens & doc_set)
        union = len(query_tokens | doc_set)
        coef = intersection / union
        
        if coef > 0: scores.append((i, coef))
    return sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]

# --- B. TF-IDF (Vectorial con Log-Normalización) ---
def obtener_vocabulario(corpus):
    vocab = set()
    for doc in corpus: vocab.update(doc)
    return sorted(list(vocab))

def calcular_idf(corpus, vocab):
    N = len(corpus)
    idf = {}
    df_counts = Counter()
    for doc in corpus: df_counts.update(set(doc))
    for word, count in df_counts.items():
        # IDF Estándar base 10
        idf[word] = math.log10(N / (count + 1))
    return idf

def generar_matriz_tfidf(corpus, vocab, idf_dict):
    matriz = []
    for doc in corpus:
        tf_dict = Counter(doc)
        vector = []
        for word in vocab:
            raw_tf = tf_dict[word]
            # Suavizado Logarítmico: 1 + log(tf)
            tf = (1 + math.log(raw_tf)) if raw_tf > 0 else 0
            vector.append(tf * idf_dict.get(word, 0))
        matriz.append(vector)
    return pd.DataFrame(matriz, columns=vocab)

def busqueda_tfidf(consulta, matriz_tfidf, vocab, idf_dict, top_n=10):
    tokens = procesar_texto_seguro(consulta)
    q_counts = Counter(tokens)
    vec_q = []
    for word in vocab:
        raw_tf = q_counts[word]
        tf = (1 + math.log(raw_tf)) if raw_tf > 0 else 0
        vec_q.append(tf * idf_dict.get(word, 0))
    vec_q = np.array(vec_q)
    
    norm_q = norm(vec_q)
    if norm_q == 0: return []
    
    scores = []
    for i, row in matriz_tfidf.iterrows():
        vec_d = np.array(row)
        norm_d = norm(vec_d)
        dot = np.dot(vec_q, vec_d)
        sim = dot / (norm_q * norm_d) if norm_d > 0 else 0
        if sim > 0: scores.append((i, sim))
    return sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]

# --- C. BM25 (Probabilístico) ---
def obtener_estadisticas_bm25(corpus):
    doc_lens = [len(d) for d in corpus]
    return doc_lens, sum(doc_lens) / len(corpus)

def calcular_idf_bm25(corpus):
    N = len(corpus)
    idf = {}
    df_counts = Counter()
    for doc in corpus: df_counts.update(set(doc))
    for word, count in df_counts.items():
        # IDF Probabilístico
        idf[word] = math.log((N - count + 0.5) / (count + 0.5) + 1)
    return idf

def busqueda_bm25(consulta, corpus, doc_lens, avgdl, idf_bm25, k1=1.8, b=0.9, top_n=10):
    q_tokens = procesar_texto_seguro(consulta)
    scores = []
    for i, doc in enumerate(corpus):
        score = 0
        doc_counts = Counter(doc)
        for token in q_tokens:
            if token in doc_counts:
                tf = doc_counts[token]
                numerador = tf * (k1 + 1)
                denominador = tf + k1 * (1 - b + b * (doc_lens[i] / avgdl))
                score += idf_bm25.get(token, 0) * (numerador / denominador)
        if score > 0: scores.append((i, score))
    return sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]

print("✅ Modelos definidos correctamente.")

✅ Modelos definidos correctamente.


In [99]:
# =============================================================================
# 3. CARGA DE DATOS CRANFIELD
# =============================================================================

path_docs_folder = "/kaggle/input/cranfield-dataset/Cranfield"
path_query_file = "/kaggle/input/cranfield-dataset/TEST/query.txt"
path_qrels_folder = "/kaggle/input/cranfield-dataset/TEST/RES"

# 1. PARSER DE DOCUMENTOS
def cargar_docs_cranfield(ruta_carpeta):
    data = {}
    if not os.path.exists(ruta_carpeta): return data
    archivos = os.listdir(ruta_carpeta)
    print(f"Cargando documentos desde {ruta_carpeta}...")
    for archivo in archivos:
        if archivo.endswith(".txt"):
            try:
                doc_id = int(archivo.replace('.txt', ''))
                with open(os.path.join(ruta_carpeta, archivo), 'r', encoding='utf-8', errors='ignore') as f:
                    data[doc_id] = {'T': '', 'W': f.read()}
            except: continue
    return data

# 2. PARSER DE CONSULTAS (Línea por línea)
def cargar_queries_cranfield(ruta_archivo):
    data = {}
    if not os.path.exists(ruta_archivo): return data
    print(f"Cargando consultas desde {ruta_archivo}...")
    with open(ruta_archivo, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
    for idx, line in enumerate(lines):
        line = line.strip()
        if len(line) > 5:
            data[idx + 1] = {'W': line}
    return data

# 3. PARSER DE QRELS
def cargar_qrels_cranfield(ruta_carpeta):
    qrels_list = []
    if not os.path.exists(ruta_carpeta): return pd.DataFrame()
    print(f"Cargando Ground Truth (QRELS)...")
    archivos = os.listdir(ruta_carpeta)
    for archivo in archivos:
        if archivo.endswith(".txt"):
            try:
                query_id = int(archivo.replace('.txt', ''))
                with open(os.path.join(ruta_carpeta, archivo), 'r') as f:
                    content = f.read()
                numeros = content.replace('\n', ' ').split()
                for item in numeros:
                    if item.isdigit():
                        qrels_list.append([query_id, int(item), 1])
            except: continue
    return pd.DataFrame(qrels_list, columns=['query_id', 'document_id', 'relevance'])

print("INICIANDO SISTEMA...")
documents = cargar_docs_cranfield(path_docs_folder)
queries = cargar_queries_cranfield(path_query_file)
qrels_df = cargar_qrels_cranfield(path_qrels_folder)

print(f"\nESTADÍSTICAS DEL DATASET:")
print(f"   -> Documentos: {len(documents)}")
print(f"   -> Consultas: {len(queries)}")
print(f"   -> Qrels: {len(qrels_df)}")

# --- CONSTRUCCIÓN DEL CORPUS ---
if len(documents) > 0:
    print("\nProcesando corpus (Limpieza + Stemming)...")
    corpus_tokens = []
    map_indices_id = [] 
    
    for doc_id in sorted(documents.keys()):
        data = documents[doc_id]
        texto = f"{data.get('T', '')} {data.get('W', '')}"
        corpus_tokens.append(procesar_texto_seguro(texto))
        map_indices_id.append(doc_id)

    print("Generando índices invertidos...")
    vocab = obtener_vocabulario(corpus_tokens)
    idf_tfidf = calcular_idf(corpus_tokens, vocab)
    matriz_tfidf = generar_matriz_tfidf(corpus_tokens, vocab, idf_tfidf)
    doc_lengths, avgdl = obtener_estadisticas_bm25(corpus_tokens)
    idf_bm25 = calcular_idf_bm25(corpus_tokens)
    
    print("SISTEMA LISTO.")
else:
    print("ERROR: No se cargaron documentos.")

INICIANDO SISTEMA...
Cargando documentos desde /kaggle/input/cranfield-dataset/Cranfield...
Cargando consultas desde /kaggle/input/cranfield-dataset/TEST/query.txt...
Cargando Ground Truth (QRELS)...

ESTADÍSTICAS DEL DATASET:
   -> Documentos: 1400
   -> Consultas: 225
   -> Qrels: 5285

Procesando corpus (Limpieza + Stemming)...
Generando índices invertidos...
SISTEMA LISTO.


In [100]:
def evaluar_sistema_completo(nombre_modelo):
    print(f"\n--- Evaluando Modelo: {nombre_modelo} ---")
    aps, precisions, recalls = [], [], []
    
    for q_id, q_data in queries.items():
        if q_id not in qrels_df['query_id'].values: continue
        relevantes = set(qrels_df[qrels_df['query_id'] == q_id]['document_id'].values)
        
        # BÚSQUEDA (Top 200-300 para asegurar buen cálculo de MAP)
        if nombre_modelo == 'Jaccard':
            ranking = busqueda_jaccard(q_data['W'], corpus_tokens, top_n=200)
        elif nombre_modelo == 'TF-IDF':
            ranking = busqueda_tfidf(q_data['W'], matriz_tfidf, vocab, idf_tfidf, top_n=200)
        elif nombre_modelo == 'BM25':
            ranking = busqueda_bm25(q_data['W'], corpus_tokens, doc_lengths, avgdl, idf_bm25, 
                                    k1=1.8, b=0.9, top_n=200)
            
        # Métricas (MAP)
        hits, sum_precisions = 0, 0
        for i, (idx, _) in enumerate(ranking):
            if map_indices_id[idx] in relevantes:
                hits += 1
                sum_precisions += hits / (i + 1)
        ap = sum_precisions / len(relevantes) if relevantes else 0
        aps.append(ap)
        
        # Métricas (@10)
        top_10 = ranking[:10]
        hits_10 = sum(1 for idx, _ in top_10 if map_indices_id[idx] in relevantes)
        precisions.append(hits_10 / 10.0)
        recalls.append(hits_10 / len(relevantes) if len(relevantes) > 0 else 0)

    print(f"Resultados Globales {nombre_modelo}:")
    print(f"-> MAP:          {np.mean(aps):.4f}")
    print(f"-> Precision@10: {np.mean(precisions):.4f}")
    print(f"-> Recall@10:    {np.mean(recalls):.4f}")

In [101]:
# =============================================================================
# DEMOSTRACIÓN TÉCNICA (Requerimientos A y B)
# =============================================================================

def mostrar_cumplimiento_req_a():
    print("\n" + "="*70)
    print("REQUERIMIENTO A: PROCESAMIENTO DE TEXTO (Limpieza + Stemming)")
    print("="*70)
    print("Objetivo: Mostrar la transformación de texto crudo a tokens procesados.\n")
    
    doc_id_demo = 1
    if doc_id_demo not in documents: doc_id_demo = list(documents.keys())[0]
    texto_original = documents[doc_id_demo]['W']
    
    print(f"1. TEXTO ORIGINAL (Raw Input):")
    print(f"'{texto_original[:150]}...'\n")
    
    # Simulación visual de pasos intermedios
    texto_limpio = re.sub(r"[^a-z0-9\s-]", ' ', texto_original.lower())
    print(f"2. LIMPIEZA (Regex + Guiones preservados):")
    print(f"'{texto_limpio[:150]}...'\n")
    
    # Procesamiento real
    tokens_finales = procesar_texto_seguro(texto_original)
    
    print(f"3. TOKENIZACIÓN + STEMMING (Salida del Índice):")
    print(f"   [Tokens]: {tokens_finales[:15]}...")
    print(f"\n   Total Tokens: {len(tokens_finales)} (Originales: {len(texto_original.split())})")
    print("   Nota: Se eliminaron stopwords de ingeniería ('experimental', 'investigation') y se normalizó.")

def mostrar_cumplimiento_req_b():
    print("\n" + "="*70)
    print("REQUERIMIENTO B: COMPARATIVA DE MODELOS")
    print("="*70)
    
    consulta_demo = "boundary layer flow separation"
    print(f"Consulta de Prueba: '{consulta_demo}'\n")
    
    # Top 1 de cada modelo
    res_jaccard = busqueda_jaccard(consulta_demo, corpus_tokens, top_n=1)
    res_tfidf = busqueda_tfidf(consulta_demo, matriz_tfidf, vocab, idf_tfidf, top_n=1)
    res_bm25 = busqueda_bm25(consulta_demo, corpus_tokens, doc_lengths, avgdl, idf_bm25, top_n=1)
    
    print(f"{'MODELO':<10} | {'DOC ID':<8} | {'SCORE':<10} | {'CONTENIDO'}")
    print("-" * 80)
    
    def imprimir(nombre, res):
        if res:
            idx, score = res[0]
            doc_id = map_indices_id[idx]
            txt = documents[doc_id]['W'][:40].replace('\n', ' ') + "..."
            print(f"{nombre:<10} | {doc_id:<8} | {score:.4f}     | {txt}")
        else:
            print(f"{nombre:<10} | {'---':<8} | {'0.0000':<10} | ---")

    imprimir("Jaccard", res_jaccard)
    imprimir("TF-IDF", res_tfidf)
    imprimir("BM25", res_bm25)
    print("\nConclusión: BM25 demuestra mayor discriminación gracias a su función de saturación.")

# Ejecutar demos
mostrar_cumplimiento_req_a()
mostrar_cumplimiento_req_b()


REQUERIMIENTO A: PROCESAMIENTO DE TEXTO (Limpieza + Stemming)
Objetivo: Mostrar la transformación de texto crudo a tokens procesados.

1. TEXTO ORIGINAL (Raw Input):
'experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order...'

2. LIMPIEZA (Regex + Guiones preservados):
'experimental investigation of the aerodynamics of a wing in a slipstream   an experimental study of a wing in a propeller slipstream was made in order...'

3. TOKENIZACIÓN + STEMMING (Salida del Índice):
   [Tokens]: ['aerodynam', 'wing', 'slipstream', 'wing', 'propel', 'slipstream', 'made', 'order', 'spanwis', 'distribut', 'lift', 'increas', 'due', 'slipstream', 'differ']...

   Total Tokens: 64 (Originales: 145)
   Nota: Se eliminaron stopwords de ingeniería ('experimental', 'investigation') y se normalizó.

REQUERIMIENTO B: COMPARATIVA DE MODELOS
Consulta de Prueba: 'boundary layer flow separation'

MODELO     | DOC ID   

In [105]:
def comparar_modelos_visual(query_id_prueba=1):
    """
    Ejecuta Jaccard, TF-IDF y BM25 para una misma consulta y muestra
    cuáles aciertan (✅) y cuáles fallan (❌) según las QRELS.
    """
    # 1. Obtener datos de la consulta
    if query_id_prueba not in queries:
        print("ID de consulta no encontrado.")
        return

    texto_consulta = queries[query_id_prueba]['W']
    print(f"🔎 CONSULTA ID {query_id_prueba}:")
    print(f"'{texto_consulta[:100]}...'\n")
    
    # 2. Obtener la 'Hoja de Respuestas' (QRELS)
    relevantes_reales = set(qrels_df[qrels_df['query_id'] == query_id_prueba]['document_id'].values)
    print(f"📄 Documentos Relevantes Totales en QRELS: {len(relevantes_reales)}")
    print("-" * 60)

    # --- FUNCIÓN AUXILIAR PARA IMPRIMIR RESULTADOS ---
    def mostrar_ranking(nombre, resultados):
        print(f"\n>> {nombre} (Top 10):")
        hits = 0
        for i, (idx, score) in enumerate(resultados[:10]): # Solo mostramos Top 5 para no saturar
            id_real = map_indices_id[idx]
            titulo = documents[id_real].get('T', 'Sin título').strip()[:50]
            
            # EL MOMENTO DE LA VERDAD:
            if id_real in relevantes_reales:
                marca = "✅ ACERTO"
                hits += 1
            else:
                marca = "❌ FALLO"
            
            print(f"   {i+1}. [Doc {id_real}] {marca} | Score: {score:.4f} | {titulo}...")
        print(f"   RESUMEN: {hits}/10 relevantes encontrados.")

    # 3. EJECUTAR LOS 3 MODELOS
    
    # A. JACCARD
    res_jaccard = busqueda_jaccard(texto_consulta, corpus_tokens, top_n=10)
    mostrar_ranking("MODELO 1: JACCARD (Binario)", res_jaccard)
    
    # B. TF-IDF
    res_tfidf = busqueda_tfidf(texto_consulta, matriz_tfidf, vocab, idf_tfidf, top_n=10)
    mostrar_ranking("MODELO 2: TF-IDF (Vectorial)", res_tfidf)
    
    # C. BM25
    res_bm25 = busqueda_bm25(texto_consulta, corpus_tokens, doc_lengths, avgdl, idf_bm25, top_n=10)
    mostrar_ranking("MODELO 3: BM25 (Probabilístico)", res_bm25)
    
    print("\n" + "="*60)

# --- PRUEBA CON UNA CONSULTA DIFÍCIL ---
# La consulta 1 es buena, pero prueba también la 3 o la 10
comparar_modelos_visual(20)

🔎 CONSULTA ID 20:
'20	has anyone formally determined the influence of joule heating,  produced by the induced current, ...'

📄 Documentos Relevantes Totales en QRELS: 12
------------------------------------------------------------

>> MODELO 1: JACCARD (Binario) (Top 10):
   1. [Doc 407] ✅ ACERTO | Score: 0.1429 | ...
   2. [Doc 500] ✅ ACERTO | Score: 0.1250 | ...
   3. [Doc 268] ✅ ACERTO | Score: 0.1020 | ...
   4. [Doc 963] ❌ FALLO | Score: 0.0968 | ...
   5. [Doc 269] ✅ ACERTO | Score: 0.0930 | ...
   6. [Doc 450] ❌ FALLO | Score: 0.0882 | ...
   7. [Doc 1158] ❌ FALLO | Score: 0.0882 | ...
   8. [Doc 88] ✅ ACERTO | Score: 0.0862 | ...
   9. [Doc 1008] ❌ FALLO | Score: 0.0857 | ...
   10. [Doc 270] ✅ ACERTO | Score: 0.0833 | ...
   RESUMEN: 6/10 relevantes encontrados.

>> MODELO 2: TF-IDF (Vectorial) (Top 10):
   1. [Doc 500] ✅ ACERTO | Score: 0.4689 | ...
   2. [Doc 450] ❌ FALLO | Score: 0.1880 | ...
   3. [Doc 87] ✅ ACERTO | Score: 0.1796 | ...
   4. [Doc 407] ✅ ACERTO | Score: 0.

In [103]:
def main():
    while True:
        print("\n" + "="*50)
        print(" MOTOR DE BÚSQUEDA CRANFIELD (AERODINÁMICA)")
        print("="*50)
        print("1. Consultar (Jaccard)")
        print("2. Consultar (TF-IDF)")
        print("3. Consultar (BM25)")
        print("4. Evaluar Sistema Completo")
        print("5. Salir")
        
        opc = input("\nOpción: ")
        
        if opc == '5': 
            print("Saliendo...")
            break
        elif opc == '4':
            evaluar_sistema_completo('Jaccard')
            evaluar_sistema_completo('TF-IDF')
            evaluar_sistema_completo('BM25')
        elif opc in ['1', '2', '3']:
            consulta = input("Consulta: ")
            print(f"Buscando: '{consulta}'...")
            
            res = []
            if opc == '1': res = busqueda_jaccard(consulta, corpus_tokens)
            elif opc == '2': res = busqueda_tfidf(consulta, matriz_tfidf, vocab, idf_tfidf)
            elif opc == '3': res = busqueda_bm25(consulta, corpus_tokens, doc_lengths, avgdl, idf_bm25)
            
            print("\n--- Resultados ---")
            for i, (idx, score) in enumerate(res):
                doc_id = map_indices_id[idx]
                prev = documents[doc_id]['W'][:80].replace('\n', ' ')
                print(f"{i+1}. [Doc {doc_id}] (Score {score:.4f}): {prev}...")

if __name__ == "__main__":
    main()


 MOTOR DE BÚSQUEDA CRANFIELD (AERODINÁMICA)
1. Consultar (Jaccard)
2. Consultar (TF-IDF)
3. Consultar (BM25)
4. Evaluar Sistema Completo
5. Salir



Opción:  4



--- Evaluando Modelo: Jaccard ---
Resultados Globales Jaccard:
-> MAP:          0.1767
-> Precision@10: 0.2071
-> Recall@10:    0.2016

--- Evaluando Modelo: TF-IDF ---
Resultados Globales TF-IDF:
-> MAP:          0.2548
-> Precision@10: 0.2862
-> Recall@10:    0.2721

--- Evaluando Modelo: BM25 ---
Resultados Globales BM25:
-> MAP:          0.2706
-> Precision@10: 0.2956
-> Recall@10:    0.2799

 MOTOR DE BÚSQUEDA CRANFIELD (AERODINÁMICA)
1. Consultar (Jaccard)
2. Consultar (TF-IDF)
3. Consultar (BM25)
4. Evaluar Sistema Completo
5. Salir


KeyboardInterrupt: Interrupted by user